In [ ]:
import os
from DUAO.src import calibration, photometry, utils
import matplotlib.pyplot as plt
import numpy as np
from astropy.io import fits
from astropy.stats import sigma_clipped_stats
from scipy.ndimage import median_filter

# Data reduction

In [ ]:
asteroids_dir = os.path.join(utils.C2PU_DATA_DIR, utils.DIR_PHOTOM, utils.DIR_ASTER)
asteroids_dir = os.path.join(asteroids_dir, os.listdir(asteroids_dir)[0])
sorted(os.listdir(asteroids_dir))

In [ ]:
fits_sci_image = os.path.join(asteroids_dir, sorted(os.listdir(asteroids_dir))[0])
fits_sci_image = os.path.abspath(fits_sci_image)
os.path.isfile(fits_sci_image)

In [ ]:
calibration.get_infos_from_image(fits_sci_image)

In [ ]:
bias_dir, darks_dir, flats_dir = utils.get_calib_dirs_photometry(fits_sci_image)
print(bias_dir, darks_dir, flats_dir)

In [ ]:
# Get image directory, failename and extension
sc_im_dir = os.path.abspath(os.path.dirname(fits_sci_image))
sc_im_name, sc_im_ext = os.path.splitext(os.path.basename(fits_sci_image))
print(sc_im_dir, sc_im_name, sc_im_ext)

In [ ]:
# Get information from FITS header
sc_date, sc_scope, sc_cam, sc_filter, sc_expos, sc_x, sc_y = calibration.get_infos_from_image(fits_sci_image)

In [ ]:
bias_list = calibration.load_bias_frames(bias_dir, sc_date, sc_cam, sc_x, sc_y)
print(bias_list)
MASTER_BIAS = calibration.master_bias(bias_list)

In [ ]:
bkg_mean, bkg_median, bkg_sigma = sigma_clipped_stats(MASTER_BIAS["data"], sigma=3.0)

plt.imshow(MASTER_BIAS["data"], cmap="Greys", vmin=bkg_median - 5 * bkg_sigma, vmax=bkg_median + 5 * bkg_sigma)
plt.colorbar()

In [ ]:
# Master dark
# TBD: check if there is already one that works
darks_list = calibration.load_dark_frames(darks_dir, sc_date, sc_cam, sc_expos, sc_x, sc_y)

In [ ]:
MASTER_DARK, HOT_PIXELS = calibration.master_dark(darks_list, use_bias=False, master_bias=MASTER_BIAS["path"])

In [ ]:
bkg_mean, bkg_median, bkg_sigma = sigma_clipped_stats(MASTER_DARK["data"], sigma=3.0)

plt.imshow(MASTER_DARK["data"], cmap="Greys", vmin=bkg_median - 5 * bkg_sigma, vmax=bkg_median + 5 * bkg_sigma)
plt.colorbar()

In [ ]:
# Master flat
# TBD: check if there is already one that works
flats_list = calibration.load_flat_frames(flats_dir, sc_date, sc_cam, sc_filter, sc_x, sc_y, override_date_check=True)
MASTER_FLAT, DEAD_PIXELS = calibration.master_flat(flats_list, MASTER_DARK["path"])

In [ ]:
bkg_mean, bkg_median, bkg_sigma = sigma_clipped_stats(MASTER_FLAT["data"], sigma=3.0)

plt.imshow(MASTER_FLAT["data"], cmap="Greys", vmin=bkg_median - 5 * bkg_sigma, vmax=bkg_median + 5 * bkg_sigma)
plt.colorbar()

In [ ]:
additive_corr = MASTER_DARK["data"]  # -MASTER_BIAS["data"]

sc_hdu = fits.open(fits_sci_image)[0]
try:
    RED_SCIENCE = (sc_hdu.data - additive_corr) / MASTER_FLAT["data"]
except ValueError:
    RED_SCIENCE = (np.transpose(sc_hdu.data) - additive_corr) / MASTER_FLAT["data"]

# Clean bad pixels
smoothed = median_filter(RED_SCIENCE, size=(5, 5))

# Hot pixels
try:
    hot_pixel = np.where(HOT_PIXELS["data"] == 1)
    RED_SCIENCE[hot_pixel] = smoothed[hot_pixel]
except:
    print("Cannot clean hot pixels")

# Dead pixels
try:
    dead_pixel = np.where(DEAD_PIXELS["data"] == 1)
    RED_SCIENCE[dead_pixel] = smoothed[dead_pixel]
except:
    print("Cannot clean dead pixels")

In [ ]:
bkg_mean, bkg_median, bkg_sigma = sigma_clipped_stats(sc_hdu.data, sigma=3.0)

plt.imshow(sc_hdu.data, cmap="Greys", vmin=bkg_median - 5 * bkg_sigma, vmax=bkg_median + 5 * bkg_sigma)
plt.colorbar()

In [ ]:
bkg_mean, bkg_median, bkg_sigma = sigma_clipped_stats(RED_SCIENCE, sigma=3.0)

plt.imshow(RED_SCIENCE, cmap="Greys", vmin=bkg_median - 5 * bkg_sigma, vmax=bkg_median + 5 * bkg_sigma)
plt.colorbar()

In [ ]:
# Write appropriate FITS files
new_fn = f"{sc_im_name}_REDUCED{sc_im_ext}"
red_hdu = sc_hdu.copy()
red_hdu.data = RED_SCIENCE
red_hdu.header["PROCTYPE"] = "RED     "
red_hdu.header["FILENAME"] = new_fn
red_hdu.header["CREATOR"] = "JOCHEVAL"
red_hdu.header["MDARK"] = MASTER_DARK["path"]
red_hdu.header["MFLAT"] = MASTER_FLAT["path"]
red_hdu.header["HPIXELS"] = HOT_PIXELS["path"]
red_hdu.header["DPIXELS"] = DEAD_PIXELS["path"]

redim_dir = os.path.join(os.path.abspath(sc_im_dir), "REDUCED")
if not os.path.isdir(redim_dir):
    os.makedirs(redim_dir)
write_path = os.path.join(redim_dir, new_fn)
red_hdu.writeto(write_path, overwrite=True)
print(f"Calibrated image written to {write_path}.")

dic_to_return = {"path": write_path, "data": RED_SCIENCE, "MASTER DARK": MASTER_DARK, "MASTER FLAT": MASTER_FLAT, "HOT PIXELS": HOT_PIXELS, "DEAD PIXELS": DEAD_PIXELS}

# Photometry

In [ ]:
# fits_sci_image = os.path.join('TD_Photometry_python', 'data', 'test_image.fits')
# fits_sci_image = os.path.abspath(fits_sci_image)

fits_sci_image = dic_to_return["path"]
sources = photometry.detect_sources(fits_sci_image)

In [ ]:
sources

In [ ]:
fwhm = photometry.get_fwhm(fits_sci_image, sources)

In [ ]:
phot_table = photometry.apert_photometry(fits_sci_image, sources, fwhm)

In [ ]:
phot_table

In [ ]:
phot_sso_table = photometry.query_sso_photometry(fits_sci_image, fwhm)

In [ ]:
phot_sso_table